In [1]:
import sys
import os
import pandas as pd 
import numpy as np
import os 
import sys 
from code.data_processing import get_all_datasets
import matplotlib.pyplot as plt
from TaskAnalysis import TaskAnalysis
from ResultHandling import ResultHandling


In [2]:
# ORMOGP 
if 0:
    experiment1 = { # get gp and bagp gp results in for comparison
        'experiment_name' : 'divmogp_experiment',
        'name_in_results_finished' : 'ORMOGP',
        'jobid' : '4082885',
        'models_exclude' : []
    }
    experiments = [experiment1]
    analysis = TaskAnalysis(experiments=experiments, save_name='4082885')
    #analysis.save_diversity_table()
    analysis.create_parameter_table()
    analysis.save_boxplots()
    #analysis.save_tables()

In [3]:
if 1:
    experiment1 = { # get gp and bagp gp results in for comparison
        'experiment_name' : 'bagboost_experiment',
        'name_in_results_finished' : 'bagboost',
        'jobid' : '4083559',
        'models_exclude' : ['nichgp', 'ccgp']
    }
    experiment2 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'fastbag',
        'name_in_results_finished' : 'fastbag_ham',
        'jobid' : '4083560',
        'models_exclude' : '',
        'name_as' : ['brf', 'brfh' ]
    }
    experiment3 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'fastbag',
        'name_in_results_finished' : 'fastbag_ham_new',
        'jobid' : '4084319',
        'models_exclude' : '',
        'name_as' : ['2brf', '2brfh' ]
    }
    experiments = [experiment1, experiment2, experiment3]
    analysis = TaskAnalysis(experiments=experiments, save_name='fastbag_ham')
    analysis.save_diversity_table()
    analysis.create_parameter_table()
    analysis.save_boxplots()
    analysis.save_tables()

AxisError: axis 2 is out of bounds for array of dimension 2

In [4]:
# Analysis new mogp
if 0:
    experiment1 = { # get gp and bagp gp results in for comparison
        'experiment_name' : 'bagboost_experiment',
        'name_in_results_finished' : 'bagboost',
        'jobid' : '4083559',
        'models_exclude' : ['nichgp', 'ccgp']
    }
    experiment2 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'bagmogp',
        'name_in_results_finished' : 'bagmogp',
        'jobid' : '4083783',
        'models_exclude' : ''
    }
    experiment3 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'bagmogp',
        'name_in_results_finished' : 'bagmogpnew',
        'jobid' : '4083783',
        'models_exclude' : ''
    }
    experiments = [experiment1, experiment2]#, experiment3]
    analysis = TaskAnalysis(experiments=experiments, save_name='bagmogpnew')
    analysis.save_diversity_table()
    analysis.create_parameter_table()
    analysis.save_boxplots()
    analysis.save_tables()
    


# Analysis BagBoost

In [5]:
if 0:
    experiment1 = {
        'experiment_name' : 'bagboost_experiment',
        'name_in_results_finished' : 'bagboost',
        'jobid' : '4083559',
        'models_exclude' : ''
    }
    experiments = [experiment1]
    analysis = TaskAnalysis(experiments=experiments, save_name='bagboost')
    analysis.save_diversity_table()
    analysis.create_parameter_table()
    analysis.save_boxplots()
    analysis.save_tables()

# Analysis RFBAG


In [6]:
if 0:
    experiment1 = { # get gp and bagp gp results in for comparison
        'experiment_name' : 'bagboost_experiment',
        'name_in_results_finished' : 'bagboost',
        'jobid' : '4083559',
        'models_exclude' : ['nichgp', 'ccgp']
    }
    experiment2 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'fastbag',
        'name_in_results_finished' : 'fastbag_ham',
        'jobid' : '4083560',
        'models_exclude' : ''
    }
    experiment3 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'fastbag',
        'name_in_results_finished' : 'fastbag_ham_new',
        'jobid' : '4084319',
        'models_exclude' : ''
    }
    experiments = [experiment1, experiment2, experiment3]
    analysis = TaskAnalysis(experiments=experiments, save_name='fastbag_ham')
    analysis.save_diversity_table()
    analysis.create_parameter_table()
    analysis.save_boxplots()
    analysis.save_tables()

# Analysis BAGM3GP

In [13]:
if 1:
    experiment1 = { # get gp and bagp gp results in for comparison
        'experiment_name' : 'bagboost_experiment',
        'name_in_results_finished' : 'bagboost',
        'jobid' : '4083559',
        'models_exclude' : ['GP', 'nichgp', 'ccgp']
    }
    experiment2 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'm3gp_bag',
        'name_in_results_finished' : 'm3gpbag',
        'jobid' : '4082937',
        'models_exclude' : ''
    }
    experiments = [experiment1, experiment2]
    analysis = TaskAnalysis(experiments=experiments, save_name='m3gpbag')
    analysis.save_diversity_table()
    analysis.create_parameter_table()
    analysis.save_boxplots()
    analysis.save_tables()

# Analysis BAGMOGP

In [8]:
if 0:
    experiment1 = { # get gp and bagp gp results in for comparison
        'experiment_name' : 'bagboost_experiment',
        'name_in_results_finished' : 'bagboost',
        'jobid' : '4083559',
        'models_exclude' : ['gp', 'nichgp', 'ccgp'],
        
    }
    experiment2 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'bagmogp',
        'name_in_results_finished' : 'bagmogp',
        'jobid' : '4083783',
        'models_exclude' : '',
        'name_as' : ['bm250']
    }
    experiment3 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'bagmogp',
        'name_in_results_finished' : 'mogp500',
        'jobid' : '4083864',
        'models_exclude' : '',
        'name_as' : ['bm500']
    }

    experiments = [experiment1, experiment2, experiment3]
    analysis = TaskAnalysis(experiments=experiments, save_name='bagmogp')
    analysis.save_diversity_table()
    analysis.create_parameter_table()
    analysis.save_boxplots()
    analysis.save_tables()
    



# PCA analysis - parameter table requires minor editing to add number of principle components because of how i structured the experiment


In [9]:
do_pca = 0

In [10]:
# graphs 
if do_pca == 1:
    experiment1 = { # get gp and bagp gp results in for comparison
        'experiment_name' : 'bagboost_experiment',
        'name_in_results_finished' : 'bagboost',
        'jobid' : '4083559',
        'models_exclude' : ['gp', 'nichgp', 'ccgp']
    }
    experiment2 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'dr',
        'name_in_results_finished' : 'pca',
        'jobid' : '4083806',
        'models_exclude' : '',
    }
    experiment3 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'dr',
        'name_in_results_finished' : 'pcanew',
        'jobid' : '4084320',
        'models_exclude' : '',
    }
    experiments = [experiment1, experiment2, experiment3]
    analysis = TaskAnalysis(experiments=experiments, save_name='pca')
    analysis.create_parameter_table()
    analysis.save_diversity_table()
    analysis.save_boxplots()

# PCA TAble first params 

In [10]:
if do_pca == 0:
    experiment1 = { # get gp and bagp gp results in for comparison
        'experiment_name' : 'bagboost_experiment',
        'name_in_results_finished' : 'bagboost',
        'jobid' : '4083559',
        'models_exclude' : ['gp', 'nichgp', 'ccgp']
    }
    experiment2 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'dr',
        'name_in_results_finished' : 'pca',
        'jobid' : '4083806',
        'models_exclude' : '',
    }
    experiments = [experiment1, experiment2]
    analysis = TaskAnalysis(experiments=experiments, save_name='pca')
    analysis.save_tables()
    analysis.save_diversity_table()

# PCA ANALYSIS duff params

In [11]:
if do_pca == 1:
    experiment1 = { # get gp and bagp gp results in for comparison
        'experiment_name' : 'bagboost_experiment',
        'name_in_results_finished' : 'bagboost',
        'jobid' : '4083559',
        'models_exclude' : ['gp', 'nichgp', 'ccgp']
    }
    experiment3 = { # main part : fast rf no ham and fast rf ham 
        'experiment_name' : 'dr',
        'name_in_results_finished' : 'pcanew',
        'jobid' : '4084320',
        'models_exclude' : '',
    }
    experiments = [experiment1, experiment3]
    analysis = TaskAnalysis(experiments=experiments, save_name='pca2')
    analysis.save_tables()
    analysis.save_diversity_table()